In [2]:
import os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, spearmanr
from sklearn.decomposition import NMF
from statsmodels.stats.multitest import multipletests
from pybdm import BDM

# Initialize BDM object
bdm = BDM(ndim=2)

# Directories and input files
idhwt_file = "IDHWT_log_normalized.csv"
k27m_file = "K27Mreal_log_normalized.csv"
output_dir = "results"
combined_dir = os.path.join(output_dir, "combined")
idhwt_dir = os.path.join(output_dir, "idhwt")
k27m_dir = os.path.join(output_dir, "k27m")
os.makedirs(combined_dir, exist_ok=True)
os.makedirs(idhwt_dir, exist_ok=True)
os.makedirs(k27m_dir, exist_ok=True)

# Parameters
logfc_threshold = 1.0
pval_threshold = 0.05
n_components = 5  # Number of transcriptional modules
top_genes_per_module = 50
threshold = 0.5  # Single threshold for binarization

# Helper: Perform DEG Analysis
def compute_deg(df1, df2):
    genes = df1.index
    deg_results = []
    for gene in genes:
        if gene not in df2.index:
            continue
        group1 = df1.loc[gene].values
        group2 = df2.loc[gene].values
        logfc = np.log2(np.mean(group2) + 1e-9) - np.log2(np.mean(group1) + 1e-9)
        pval = ttest_ind(group1, group2, equal_var=False).pvalue
        deg_results.append({"Gene": gene, "LogFC": logfc, "Pval": pval})

    deg_df = pd.DataFrame(deg_results)
    deg_df["Adj_Pval"] = multipletests(deg_df["Pval"], method="fdr_bh")[1]
    deg_df = deg_df[
        (deg_df["Adj_Pval"] < pval_threshold) & (deg_df["LogFC"].abs() >= logfc_threshold)
    ].nlargest(100, "LogFC")
    return deg_df

# Helper: Apply NNMF and save top genes
def apply_nnmf_and_save(expression_matrix, gene_names, output_dir):
    model = NMF(n_components=n_components, init="random", random_state=42, max_iter=1000)
    W = model.fit_transform(expression_matrix)
    nnmf_results = []
    for module_idx in range(W.shape[1]):
        module_scores = W[:, module_idx]
        ranked_genes = np.argsort(module_scores)[::-1]  # Sort in descending order
        top_genes = ranked_genes[:top_genes_per_module]
        for gene_idx in top_genes:
            nnmf_results.append({
                "Module": module_idx + 1,
                "Gene": gene_names[gene_idx],
                "Score": module_scores[gene_idx]
            })

    # Save NNMF results
    nnmf_results_df = pd.DataFrame(nnmf_results)
    nnmf_results_path = os.path.join(output_dir, "nnmf_top_genes.csv")
    nnmf_results_df.to_csv(nnmf_results_path, index=False)
    return nnmf_results_df

# Helper: Extract Gene Counts
def extract_gene_counts(expression_matrix, gene_names):
    return expression_matrix.loc[gene_names].values

# Helper: Compute Spearman adjacency matrix
def compute_spearman_adjacency(data):
    adjacency = np.zeros((data.shape[0], data.shape[0]))
    for i in range(data.shape[0]):
        for j in range(i + 1, data.shape[0]):
            corr, _ = spearmanr(data[i, :], data[j, :])
            adjacency[i, j] = corr
            adjacency[j, i] = corr
    return adjacency

# Helper: Perform Node-Based Perturbation
def bdm_node_analysis(matrix):
    binary_matrix = (matrix > threshold).astype(int)
    original_bdm = bdm.bdm(binary_matrix)
    perturbed_bdm_values = []
    for i in range(binary_matrix.shape[0]):
        perturbed_matrix = binary_matrix.copy()
        perturbed_matrix[i, :] = 0
        perturbed_matrix[:, i] = 0
        perturbed_bdm_values.append(original_bdm - bdm.bdm(perturbed_matrix))
    return perturbed_bdm_values

# Helper: Perform Link-Based Perturbation
def bdm_link_analysis(matrix):
    binary_matrix = (matrix > threshold).astype(int)
    original_bdm = bdm.bdm(binary_matrix)
    perturbed_link_bdm = []
    for i in range(binary_matrix.shape[0]):
        for j in range(i + 1, binary_matrix.shape[0]):
            perturbed_matrix = binary_matrix.copy()
            perturbed_matrix[i, j] = 0
            perturbed_matrix[j, i] = 0
            perturbed_link_bdm.append({
                "Link": f"{i}-{j}",
                "Perturbation_Score": original_bdm - bdm.bdm(perturbed_matrix)
            })

    return pd.DataFrame(perturbed_link_bdm)

# DEG Analysis for Combined Dataset
idhwt_df = pd.read_csv(idhwt_file, index_col=0)
k27m_df = pd.read_csv(k27m_file, index_col=0)
combined_deg = compute_deg(idhwt_df, k27m_df)
combined_deg.to_csv(os.path.join(combined_dir, "top_100_degs.csv"), index=False)

# Process Each Dataset (IDHWT, K27M) for NNMF, Centrality, Perturbations
for file, output_dir in zip([idhwt_file, k27m_file], [idhwt_dir, k27m_dir]):
    df = pd.read_csv(file, index_col=0)
    gene_names = df.index.values
    expression_matrix = df.values

    # Apply NNMF and Save Results
    nnmf_results_df = apply_nnmf_and_save(expression_matrix, gene_names, output_dir)

    # Extract counts for NNMF genes
    top_genes = nnmf_results_df["Gene"].unique()
    top_gene_counts = extract_gene_counts(df, top_genes)

    # Compute Spearman adjacency matrix
    spearman_adjacency = compute_spearman_adjacency(top_gene_counts)

    # Node Perturbation (Spearman)
    node_perturbed = bdm_node_analysis(spearman_adjacency)
    pd.DataFrame({"Gene": top_genes, "Perturbed_BDM": node_perturbed}).to_csv(
        os.path.join(output_dir, "node_perturbation_spearman.csv"), index=False
    )

    # Link Perturbation (Spearman)
    link_perturbed = bdm_link_analysis(spearman_adjacency)
    link_perturbed.to_csv(os.path.join(output_dir, "link_perturbation_spearman.csv"), index=False)

print("Analysis completed for all datasets.")


Analysis completed for all datasets.


In [8]:
import os
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from sklearn.decomposition import NMF

# Directories and input files
idhwt_file = "IDHWT_log_normalized.csv"
k27m_file = "K27Mreal_log_normalized.csv"
output_dir = "results"
idhwt_spearman_dir = os.path.join(output_dir, "idhwt_spearman")
k27m_spearman_dir = os.path.join(output_dir, "k27m_spearman")

# Create directories for Spearman results
os.makedirs(idhwt_spearman_dir, exist_ok=True)
os.makedirs(k27m_spearman_dir, exist_ok=True)

# Parameters
n_components = 5  # Number of transcriptional modules
top_genes_per_module = 50
threshold = 0.5  # Single threshold for binarization

# Helper: Apply NNMF and save top genes
def apply_nnmf_and_save(expression_matrix, gene_names, output_dir):
    model = NMF(n_components=n_components, init="random", random_state=42, max_iter=1000)
    W = model.fit_transform(expression_matrix)
    nnmf_results = []
    for module_idx in range(W.shape[1]):
        module_scores = W[:, module_idx]
        ranked_genes = np.argsort(module_scores)[::-1]  # Sort in descending order
        top_genes = ranked_genes[:top_genes_per_module]
        for gene_idx in top_genes:
            nnmf_results.append({
                "Module": module_idx + 1,
                "Gene": gene_names[gene_idx],
                "Score": module_scores[gene_idx]
            })

    # Save NNMF results
    nnmf_results_df = pd.DataFrame(nnmf_results)
    nnmf_results_path = os.path.join(output_dir, "nnmf_top_genes.csv")
    nnmf_results_df.to_csv(nnmf_results_path, index=False)
    return nnmf_results_df

# Helper: Extract Gene Counts
def extract_gene_counts(expression_matrix, gene_names):
    return expression_matrix.loc[gene_names].values

# Helper: Compute Spearman adjacency matrix
def compute_spearman_adjacency(data):
    adjacency = np.zeros((data.shape[0], data.shape[0]))
    for i in range(data.shape[0]):
        for j in range(i + 1, data.shape[0]):
            corr, _ = spearmanr(data[i, :], data[j, :])
            adjacency[i, j] = corr
            adjacency[j, i] = corr
    return adjacency

# Helper: Perform Link-Based Perturbation (Spearman) with Gene Names
def spearman_link_analysis_with_gene_names(matrix, gene_names):
    binary_matrix = (matrix > threshold).astype(int)
    perturbed_link_spearman = []
    for i in range(binary_matrix.shape[0]):
        for j in range(i + 1, binary_matrix.shape[0]):
            perturbed_matrix = binary_matrix.copy()
            perturbed_matrix[i, j] = 0
            perturbed_matrix[j, i] = 0
            perturbation_score = np.sum(np.abs(matrix - perturbed_matrix))  # Custom metric
            perturbed_link_spearman.append({
                "Link": f"{gene_names[i]}-{gene_names[j]}",
                "Perturbation_Score": perturbation_score
            })
    return pd.DataFrame(perturbed_link_spearman)

# Process Each Dataset (IDHWT, K27M) for NNMF, Spearman Link Perturbations
for file, output_dir in zip([idhwt_file, k27m_file], [idhwt_spearman_dir, k27m_spearman_dir]):
    df = pd.read_csv(file, index_col=0)
    gene_names = df.index.values
    expression_matrix = df.values

    # Apply NNMF and Save Results
    nnmf_results_df = apply_nnmf_and_save(expression_matrix, gene_names, output_dir)

    # Extract counts for NNMF genes
    top_genes = nnmf_results_df["Gene"].unique()
    top_gene_counts = extract_gene_counts(df, top_genes)

    # Compute Spearman adjacency matrix
    spearman_adjacency = compute_spearman_adjacency(top_gene_counts)

    # Link Perturbation (Spearman) with Gene Names
    link_perturbed = spearman_link_analysis_with_gene_names(spearman_adjacency, top_genes)
    link_perturbed.to_csv(os.path.join(output_dir, "link_perturbation_spearman_with_genes.csv"), index=False)

print("Spearman Link Perturbation Analysis with Gene Names completed for both datasets.")


Spearman Link Perturbation Analysis with Gene Names completed for both datasets.


In [ ]:
###PID Analysis 

In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from pybdm import BDM, PerturbationExperiment

# Directories and input files
idhwt_file = "IDHWT_log_normalized.csv"
k27m_file = "K27Mreal_log_normalized.csv"
output_dir = "results"
bdm_output_dir = os.path.join(output_dir, "BDM_Analysis_Results")
os.makedirs(bdm_output_dir, exist_ok=True)

# Parameters
n_components = 5  # Number of transcriptional modules
top_genes_per_module = 50

# Helper: Apply NNMF and save top genes
def apply_nnmf_and_save(expression_matrix, gene_names, output_dir):
    model = NMF(n_components=n_components, init="random", random_state=42, max_iter=1000)
    W = model.fit_transform(expression_matrix)
    nnmf_results = []
    for module_idx in range(W.shape[1]):
        module_scores = W[:, module_idx]
        ranked_genes = np.argsort(module_scores)[::-1]  # Sort in descending order
        top_genes = ranked_genes[:top_genes_per_module]
        for gene_idx in top_genes:
            nnmf_results.append({
                "Module": module_idx + 1,
                "Gene": gene_names[gene_idx],
                "Score": module_scores[gene_idx]
            })

    # Save NNMF results
    nnmf_results_df = pd.DataFrame(nnmf_results)
    nnmf_results_path = os.path.join(output_dir, "nnmf_top_genes.csv")
    nnmf_results_df.to_csv(nnmf_results_path, index=False)
    return nnmf_results_df

# Helper: Construct Binary Adjacency Matrix
def construct_binary_adjacency(data, gene_names):
    num_genes = data.shape[0]
    adjacency_matrix = np.zeros((num_genes, num_genes))

    for i in range(num_genes):
        for j in range(i + 1, num_genes):
            corr = np.corrcoef(data[i, :], data[j, :])[0, 1]
            adjacency_matrix[i, j] = 1 if corr > 0.5 else 0  # Example threshold
            adjacency_matrix[j, i] = adjacency_matrix[i, j]

    adjacency_df = pd.DataFrame(adjacency_matrix, index=gene_names, columns=gene_names)
    return adjacency_df

# Helper: BDM Perturbation Analysis
def bdm_perturbation_analysis(adj_matrix):
    """
    Perform BDM perturbation analysis on an adjacency matrix.
    """
    bdm = BDM(ndim=2)
    binary_matrix = (adj_matrix > 0).astype(int).values  # Binarize adjacency matrix
    perturbation = PerturbationExperiment(bdm, binary_matrix, metric="bdm")
    delta_bdm = perturbation.run()
    reshaped_delta_bdm = np.reshape(delta_bdm, adj_matrix.shape)
    return pd.DataFrame(reshaped_delta_bdm, index=adj_matrix.index, columns=adj_matrix.columns)

# Save Top Perturbations
def save_top_perturbations(perturbation_results, output_folder, tissue_type):
    """
    Save the top 30 gene-gene and single-node perturbations with the highest absolute values.
    """
    # Gene-Gene Perturbations
    perturbation_scores = perturbation_results.abs().unstack().sort_values(ascending=False)
    top_gene_gene = perturbation_scores.head(30)
    top_gene_gene.to_csv(os.path.join(output_folder, f"Top_30_Gene_Gene_Perturbations_{tissue_type}.csv"), header=["Perturbation_Score"])

    # Single-Node Perturbations
    single_node_scores = perturbation_results.abs().sum(axis=1)
    top_single_node = single_node_scores.nlargest(30)
    top_single_node.to_csv(os.path.join(output_folder, f"Top_30_Single_Node_Perturbations_{tissue_type}.csv"), header=["Perturbation_Score"])

# Process Each Dataset (IDHWT, K27M) for BDM-Based Adjacency, Node, and Link Perturbations
for file, tissue_type in zip([idhwt_file, k27m_file], ["IDHWT", "K27M"]):
    df = pd.read_csv(file, index_col=0)
    gene_names = df.index.values
    expression_matrix = df.values

    # Apply NNMF and Save Results
    nnmf_results_df = apply_nnmf_and_save(expression_matrix, gene_names, bdm_output_dir)

    # Extract counts for NNMF genes
    top_genes = nnmf_results_df["Gene"].unique()
    top_gene_counts = df.loc[top_genes].values

    # Construct Binary Adjacency Matrix
    binary_adjacency = construct_binary_adjacency(top_gene_counts, top_genes)

    # Perform BDM Perturbation Analysis
    perturbation_results = bdm_perturbation_analysis(binary_adjacency)

    # Save Full Perturbation Results
    perturbation_results.to_csv(os.path.join(bdm_output_dir, f"{tissue_type}_Full_Perturbation_Results.csv"))

    # Save Top Perturbations
    save_top_perturbations(perturbation_results, bdm_output_dir, tissue_type)

print("BDM-based perturbation analysis completed for all datasets.")


BDM-based perturbation analysis completed for all datasets.


In [ ]:
###NNMF analysis (unsupervised): co-expression modules

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

# Directories
input_files = {"K27M": "K27Mreal_log_normalized.csv", "IDHWT": "IDHWT_log_normalized.csv"}
output_dir = "NNMF_Clusters"
os.makedirs(output_dir, exist_ok=True)

# Subfolders
subfolders = {label: os.path.join(output_dir, label) for label in input_files.keys()}
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Extract NNMF Clusters and Save Top 30 Genes

def save_nnmf_clusters(data_file, label_name, output_folder, n_components=3):
    # Load the data
    df = pd.read_csv(data_file)
    gene_names = df.iloc[:, 0].values  # Gene names
    expression_data = df.iloc[:, 1:].values  # Gene expression data

    # Apply NNMF
    nmf = NMF(n_components=n_components, random_state=42)
    W = nmf.fit_transform(expression_data)  # Gene contributions to clusters
    H = nmf.components_  # Cluster scores

    # Save full NNMF cluster assignments
    cluster_assignments = np.argmax(W, axis=1)
    cluster_df = pd.DataFrame({"Gene": gene_names, "Cluster": cluster_assignments})
    cluster_df.to_csv(os.path.join(output_folder, f"{label_name}_nnmf_clusters.csv"), index=False)

    # Save top 30 genes for each cluster
    for cluster_idx in range(n_components):
        cluster_scores = W[:, cluster_idx]
        top_genes_idx = np.argsort(cluster_scores)[::-1][:30]  # Top 30 genes by score
        top_genes = gene_names[top_genes_idx]
        top_scores = cluster_scores[top_genes_idx]

        top_genes_df = pd.DataFrame({"Gene": top_genes, "Score": top_scores})
        top_genes_df.to_csv(
            os.path.join(output_folder, f"{label_name}_top_30_genes_cluster_{cluster_idx}.csv"),
            index=False
        )

# Run NNMF Analysis
for label, file_path in input_files.items():
    print(f"Processing {label}...")
    save_nnmf_clusters(file_path, label, subfolders[label])

print("NNMF clusters and top genes saved.")

Processing K27M...


C:\Users\uabic\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Processing IDHWT...
NNMF clusters and top genes saved.
